In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np


# Load the CSV files
amplitude_spectrum = pd.read_csv('amplitude_spectrum.csv')
decibels = pd.read_csv('delta-theta-alpha-beta-RAHC7-decibels.csv')
spectrum = pd.read_csv('delta_theta_alpha_beta-RAHC7.csv')
binary_labels = pd.read_csv('r_r_binary.csv')  # Ensure the binary labels match the samples

# Standardize each dataset
scaler = StandardScaler()

amplitude_spectrum_standardized = scaler.fit_transform(amplitude_spectrum)
decibels_standardized = scaler.fit_transform(decibels)
spectrum_standardized = scaler.fit_transform(spectrum)

# Convert binary labels to numpy array
binary_labels = binary_labels['Binary Label'].values  

2024-11-30 15:01:50.827307: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-30 15:01:50.833791: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-30 15:01:50.857410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732996910.898092   12765 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732996910.912694   12765 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-30 15:01:50.946190: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [4]:
len(binary_labels)

253

In [7]:
from sklearn.model_selection import KFold

def build_ffnn(input_dim):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(4, activation='relu', input_shape=(input_dim,)),
        tf.keras.layers.Dense(4, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model





In [8]:
from sklearn.metrics import accuracy_score

# Prepare datasets
datasets = {
    'Amplitude Spectrum': amplitude_spectrum_standardized,
    'Decibels': decibels_standardized,
    'Spectrum': spectrum_standardized
}

# Initialize 4-fold cross-validation
kf = KFold(n_splits=4, shuffle=True, random_state=42)

# Store results
results = {}

for name, data in datasets.items():
    print(f"\nPerforming 4-Fold Cross-Validation on {name} data...")
    fold_accuracies = []
    fold_losses = []
    
    # Perform cross-validation
    for fold, (train_idx, test_idx) in enumerate(kf.split(data)):
        print(f"\nFold {fold + 1}")
        
        # Split data
        X_train, X_test = data[train_idx], data[test_idx]
        y_train, y_test = binary_labels[train_idx], binary_labels[test_idx]
        
        # Build and train the model
        model = build_ffnn(input_dim=X_train.shape[1])
        history = model.fit(X_train, y_train, epochs=20, batch_size=10, verbose=0)
        
        # Evaluate the model on the test set
        loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
        fold_losses.append(loss)
        fold_accuracies.append(accuracy)
        print(f"Fold {fold + 1} - Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")
    
    # Store mean performance across folds
    results[name] = {
        'Mean Loss': np.mean(fold_losses),
        'Mean Accuracy': np.mean(fold_accuracies)
    }

# Print final results
print("\nFinal Results:")
for name, metrics in results.items():
    print(f"{name} - Mean Loss: {metrics['Mean Loss']:.4f}, Mean Accuracy: {metrics['Mean Accuracy']:.4f}")



Performing 4-Fold Cross-Validation on Amplitude Spectrum data...

Fold 1
Fold 1 - Loss: 0.6941, Accuracy: 0.4603

Fold 2
Fold 2 - Loss: 0.7009, Accuracy: 0.4286

Fold 3
Fold 3 - Loss: 0.7571, Accuracy: 0.5079

Fold 4
Fold 4 - Loss: 0.7158, Accuracy: 0.4762

Performing 4-Fold Cross-Validation on Decibels data...

Fold 1
Fold 1 - Loss: 0.7030, Accuracy: 0.4603

Fold 2
Fold 2 - Loss: 0.6900, Accuracy: 0.5556

Fold 3
Fold 3 - Loss: 0.7031, Accuracy: 0.4603

Fold 4
Fold 4 - Loss: 0.7249, Accuracy: 0.5714

Performing 4-Fold Cross-Validation on Spectrum data...

Fold 1
Fold 1 - Loss: 0.6882, Accuracy: 0.5397

Fold 2
Fold 2 - Loss: 0.6969, Accuracy: 0.4762

Fold 3
Fold 3 - Loss: 0.6942, Accuracy: 0.5238

Fold 4
Fold 4 - Loss: 0.6993, Accuracy: 0.4762

Final Results:
Amplitude Spectrum - Mean Loss: 0.7170, Mean Accuracy: 0.4683
Decibels - Mean Loss: 0.7052, Mean Accuracy: 0.5119
Spectrum - Mean Loss: 0.6947, Mean Accuracy: 0.5040
